In [ ]:
pip install scikit-learn pickle-mixin bluetarget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn import svm
from sklearn import datasets

import pickle

# Load training data set
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Train the model
clf = svm.SVC(gamma='scale')
clf.fit(X, y)

pickle.dump(clf, open('./model.pkl', 'wb'))

In [ ]:
%%writefile service.py
import os
from typing import Dict, List

class Model:
    def __init__(self) -> None:
        self._model = None

    def load(self):
        import pickle

        with open(f"{os.path.dirname(__file__)}/model.pkl", 'rb') as pickle_file:
            self._model = pickle.load(pickle_file)

    def predict(self, request: Dict) -> Dict[str, List]:
        response = {}
        inputs = request["inputs"]
        result = self._model.predict(inputs).tolist()
        response["predictions"] = result

        return response

Writing service.py


In [ ]:
%%writefile requirements.txt
scikit-learn==1.0.2
pickle-mixin==1.0.2

Writing requirements.txt


In [ ]:
from bluetarget import BlueTarget

bt = BlueTarget(api_key="YOUR_API_KEY")

bt.deploy(
    model_name="FirstModel",
    model_class="Model",
    model_files=["service.py", "model.pkl"],
    requirements_file="requirements.txt"
)